In [1]:
import gensim
import os
import jsonlines

Обрабатывает файлы, по которым строится модель word2vec. Файлы должны лежать в /корневая папка проекта/data

In [2]:
class sentences_file:
    def __init__(self, path):
        self.dirname = path


    def __iter__(self):
        for file_name in os.listdir(self.dirname):
            for obj in jsonlines.open(os.path.join(self.dirname, file_name)):
                yield obj


sentences = sentences_file(path = './data')
model = gensim.models.Word2Vec(sentences, min_count = 10)

In [3]:
from ispras import texterra


isp_api = texterra.API('a7225460c1256d265d2c675ecce7d863cda7f135')

In [4]:
import re


class article:
    def __init__(self, filedir):
        self.dirname = filedir
        self.title = filedir.split('/')[-1]
        self.text = self.get_text()        
        
        
    def to_sentences(self):
        sentences = re.split('!|\?|\.+', self.text)
        for item in sentences:
            if not item is '':
                yield item
                
                
    def get_text(self):
        body = ''
        with open(self.dirname) as infile:
            for line in infile:
                body += line
                if body[-1] == '\n':
                    body = body[:-1]
        return body

Формирование файла со ссылками на статьи, по которым ползает meduza_spider.py  

In [13]:
import vk
import jsonlines
import sys


session = vk.AuthSession(6114637, 'login', 'password', scope='groups')


vk_api = vk.API(session)
meduza_id = -76982440
post_cnt = 100

with jsonlines.open('articles_urls.jl', mode = 'w') as writer:
    for _offset in range(0, 20001, 100):
        posts = None
        while True:
            try:
                posts = vk_api.wall.get(owner_id = meduza_id, offset = _offset, count = post_cnt, filter = 'owner')[1:]
            except KeyboardInterrupt:
                sys.exit()
            except:
                continue
            else:
                break
        for post in posts:
            if post.get('post_type') == 'post':
                try:
                    props = post.get('attachments')
                    _link = props[0].get('link')
                    writer.write(_link.get('url'))
                except KeyboardInterrupt:
                    sys.exit()
                except:
                    continue

Составление словаря сленга (хранится в файле slang_dictionary.jl)

In [5]:
import jsonlines


dictionary = set()
with jsonlines.open('norm_articles.jl') as sentences:
    for lst in sentences:
        for word in lst:
            dictionary.add(word)

In [12]:
import pymorphy2 as pm


morph = pm.MorphAnalyzer()
dictionary = list(dictionary)


def choose_synonyms(item_normal, item_POS, synonyms):
    chosen_synonyms = []
    for synonym in synonyms:
        synonym_parses = morph.parse(synonym[0])
        synonym_parse = None
        for s in synonym_parses:
            if s.normal_form == synonym[0]:
                synonym_parse = s
                break
        if synonym_parse is None:
            continue
        if synonym_parse.tag.POS == item_POS and synonym_parse.normal_form not in dictionary:
                chosen_synonyms.append(synonym_parse)
    return chosen_synonyms[:]


with jsonlines.open('slang_dictionary.jl', 'w') as _dict:
    word_parse = ''
    for word in dictionary:
        try:
            model.wv[word]
        except:
            continue
        synonyms = model.most_similar(positive = [word], topn = 40)
        word_parses = morph.parse(word)
        for item in word_parses:
            if item.normal_form == word:
                word_parse = item
                break
        syn_list_parses = choose_synonyms(word, word_parse.tag.POS, synonyms)
        if len(syn_list_parses) > 0:
            syn_list = [syn.normal_form for syn in syn_list_parses]
            dict_rec = word + ': ' + ', '.join(map(str, syn_list))
            _dict.write(dict_rec)

Для того, чтобы запустить какой-нибудь пример, текст статьи нужно самостоятельно перетащить в файл и '/path/to/the/file/with/the/article' заменить на путь к этому файлу

In [40]:
import nltk


tokenizer = nltk.TweetTokenizer()
morph = pm.MorphAnalyzer()


def inflect_by_template(word, template):
    grammemes = sorted(template.tag.grammemes)
    for g in grammemes:
        w = word.inflect({g})
        if w is not None:
            word = w
    return word


_article = article(filedir = '/path/to/the/file/with/the/article')
sent_gen = _article.to_sentences()
for sent in sent_gen:
    new_sent = []
    tokens = tokenizer.tokenize(sent)
    for item in tokens:
        if not item.isalnum():
            new_sent.append(item)
            continue  
        item_parse = morph.parse(item)[0]
        item_normal = item_parse.normal_form 
        try:
            model.wv[item_normal]
        except:
            new_sent.append(item)
            continue
        item_POS = item_parse.tag.POS
        if item_POS in ["PREP", "CONJ", "PRCL"]:
            new_sent.append(item)
            continue
        synonyms = model.most_similar(positive = [item_normal], topn = 40)
        chosen_synonyms = choose_synonyms(item_normal, item_POS, synonyms)
        if len(chosen_synonyms) == 0:
            new_sent.append(item)
            continue
        chosen_synonym = inflect_by_template(chosen_synonyms[0], item_parse)
        new_sent.append(chosen_synonym.word)     
    print(sent)
    print(' '.join(new_sent))
    print('\n')

Мусульмане в Москве провели несанкционированный митинг против «буддистов-террористов»
Мусульмане в Москве провели несанкционированный митинг против « буддистов-террористов »


 Репортаж «Медузы»3 сентября несколько сотен человек провели митинг на Большой Никтиской улице в Москве — у посольства Мьянмы
киселёва « медузки » 3 ома этак сотен человек провели митинг на Большой Никтиской улице в Москве — у лесопилки Мьянмы


 Они требовали остановить геноцид исповедующей ислам этнической группы Рохинджа в Мьянме
Они требовали остановить геноцид исповедующей ислам этнической группы Рохинджа в Мьянме


 Тему притеснения мусульман в Мьянме 31 августа поднял глава Чечни Рамзан Кадыров
срача притеснения мусульман в Мьянме 31 ома поднял усср Чечни путена путена


 Акция была стихийной, но несмотря на это полиция не стала вмешиваться, лишь надежнее огородила посольство от возможных инцидентов
Акция была стихийной , но несмотря на это полиция не стала учавствовать , лишь надежнее огородила лесопилку 